## [Dacon] 제주 신용카드 빅데이터 경진대회 경진대회
## Wyatt (팀명)
## 2020년 8월 12일 (제출날짜)

## 1. 라이브러리 및 데이터
## Library & Data

In [1]:
# 라이브러리 임포트

import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

In [2]:
# 버전 확인

print('Pandas : %s'%(pd.__version__))
print('Numpy : %s'%(np.__version__))
print('Scikit-Learn : %s'%(sklearn.__version__))
!python --version

Pandas : 1.0.3
Numpy : 1.18.5
Scikit-Learn : 0.23.0
Python 3.7.1


In [29]:
# 데이터 로드

df_1 = pd.read_csv("201901-202003.csv") # 1차 공개 데이터
df_2 = pd.read_csv("202004.csv") # 2차 공개 데이터
sub = pd.read_csv("submission.csv", index_col=0) # 제출용 데이터

In [30]:
df_1.shape, df_2.shape, sub.shape

((24697792, 12), (1350322, 12), (1394, 4))

In [31]:
# 1차 공개 데이터와 2차 공개 데이터를 합쳐줍니다.

df = pd.concat([df_1, df_2])
df.shape

(26048114, 12)

## 2. 데이터 전처리
## Data Cleansing & Pre-Processing  

In [32]:
# 결측치 확인합니다.

df.isnull().sum()

REG_YYMM             0
CARD_SIDO_NM         0
CARD_CCG_NM      92371
STD_CLSS_NM          0
HOM_SIDO_NM          0
HOM_CCG_NM      155234
AGE                  0
SEX_CTGO_CD          0
FLC                  0
CSTMR_CNT            0
AMT                  0
CNT                  0
dtype: int64

In [33]:
# CARD_CCG_NM의 결측치 확인합니다.

df[df['CARD_CCG_NM'].isnull()]['CARD_SIDO_NM'].value_counts()

세종    92371
Name: CARD_SIDO_NM, dtype: int64

In [34]:
# HOM_CCG_NM의 결측치 확인합니다.

df[df['HOM_CCG_NM'].isnull()]['HOM_SIDO_NM'].value_counts()

세종    155234
Name: HOM_SIDO_NM, dtype: int64

In [6]:
# 다행히 결측치가 전부 세종입니다. 세종은 세종시밖에 없으니 세종시로 채워줍니다.

df = df.fillna('세종시')

## 3. 탐색적 자료분석
## Exploratory Data Analysis


## 4. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling  

### 4.1. INOUT 값 만들기

In [7]:
# 외부인의 결제여부를 의미하는 칼럼인 INOUT 칼럼을 만듭니다.

INOUT = []

for sido1, sido2 in zip(df['CARD_SIDO_NM'], df['HOM_SIDO_NM']):
    if sido1 == sido2:
        INOUT.append(0)
    else:
        INOUT.append(1)
        
df['INOUT'] = INOUT
df.drop(['HOM_SIDO_NM'], axis=1, inplace=True)

In [8]:
# 학습할 칼럼들을 기준으로 groupby.sum을해줍니다.

df = df.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'INOUT'])['AMT'].sum().reset_index()
df.shape

(207395, 6)

In [9]:
df.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,INOUT,AMT
0,201901,강원,강릉시,건강보조식품 소매업,0,24027180
1,201901,강원,강릉시,골프장 운영업,0,22052500
2,201901,강원,강릉시,골프장 운영업,1,3778500
3,201901,강원,강릉시,과실 및 채소 소매업,0,116767260
4,201901,강원,강릉시,과실 및 채소 소매업,1,138123811


### 4.2. 존재하지 않는 row 값 채워주기

In [10]:
# 아예 존재하지 않는 row가 있습니다.
# 아예 존재하지 않는 row를 만들어주기 위해 템플릿을 만듭니다.

columns = ['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'INOUT']

REG_YYMMs = df['REG_YYMM'].unique().tolist()
CARD_SIDO_NMs = df['CARD_SIDO_NM'].unique().tolist()
CARD_CCG_NMs = df['CARD_CCG_NM'].unique().tolist()
STD_CLSS_NMs  = df['STD_CLSS_NM'].unique().tolist()
INOUTs = [0, 1]

temp = []

for REG_YYMM in REG_YYMMs:
    for CARD_SIDO_NM in CARD_SIDO_NMs:
        for CARD_CCG_NM in CARD_CCG_NMs:
            for STD_CLSS_NM in STD_CLSS_NMs:
                for INOUT in INOUTs:
                    temp.append([REG_YYMM, CARD_SIDO_NM, CARD_CCG_NM, STD_CLSS_NM, INOUT])

temp = pd.DataFrame(data=temp, columns=columns)

In [11]:
# 만든 템플릿과 1차 전처리 결과물을 합쳐줍니다.

df = pd.merge(temp, df, how='left', on=['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'INOUT'])

In [12]:
# 빈 값은 결제액이 없는 데이터들이니 전부 0으로 채워줍니다.

df = df.fillna(0)
df.shape

(5063008, 6)

In [13]:
df.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,INOUT,AMT
0,201901,강원,강릉시,건강보조식품 소매업,0,24027180.0
1,201901,강원,강릉시,건강보조식품 소매업,1,0.0
2,201901,강원,강릉시,골프장 운영업,0,22052500.0
3,201901,강원,강릉시,골프장 운영업,1,3778500.0
4,201901,강원,강릉시,과실 및 채소 소매업,0,116767260.0


### 4.3. year, month 값 만들기

In [14]:
# 함수를 만듭니다.

def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [15]:
# REG_YYMM을 연, 월로 나눠줍니다. 기존 칼럼 드랍 해줍니다.

df['year'] = df['REG_YYMM'].apply(lambda x: grap_year(x))
df['month'] = df['REG_YYMM'].apply(lambda x: grap_month(x))
df = df.drop(['REG_YYMM'], axis=1)
df.shape

(5063008, 7)

In [16]:
df.head()

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,INOUT,AMT,year,month
0,강원,강릉시,건강보조식품 소매업,0,24027180.0,2019,1
1,강원,강릉시,건강보조식품 소매업,1,0.0,2019,1
2,강원,강릉시,골프장 운영업,0,22052500.0,2019,1
3,강원,강릉시,골프장 운영업,1,3778500.0,2019,1
4,강원,강릉시,과실 및 채소 소매업,0,116767260.0,2019,1


### 4.4. 인코딩 및 데이터 셋 분리

In [17]:
# 기계가 학습을 하기 위해 인코딩 해줍니다.

dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder

for column in encoders.keys():
    encoder = encoders[column]
    df[column] = encoder.transform(df[column])

In [18]:
# feature, target 설정합니다.

train_num = df.sample(frac=1, random_state=0)
train_features = train_num.drop(['AMT'], axis=1)
train_target = np.log1p(train_num['AMT'])

## 5. 모델 학습 및 검증
## Model Tuning & Evaluation

In [19]:
# 학습 시킵니다.

model = RandomForestRegressor(n_jobs=-1, random_state=42)
model.fit(train_features, train_target)

RandomForestRegressor(n_jobs=-1, random_state=42)

## 6. 결과 및 결언
## Conclusion & Discussion

### 06.1. 7월 예측하기

In [20]:
# 7월을 예측하기 위한 템플릿을 만들어줍니다.

columns = ['CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'INOUT', 'year', 'month']

CARD_SIDO_NMs = df['CARD_SIDO_NM'].unique().tolist()
CARD_CCG_NMs = df['CARD_CCG_NM'].unique().tolist()
STD_CLSS_NMs = df['STD_CLSS_NM'].unique().tolist()
INOUTs= [0, 1]
years = [2020]
months = [7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for CARD_CCG_NM in CARD_CCG_NMs:
        for STD_CLSS_NM in STD_CLSS_NMs:
            for INOUT in INOUTs:
                for year in years:
                    for month in months:
                        temp.append([CARD_SIDO_NM, CARD_CCG_NM, STD_CLSS_NM, INOUT, year, month])

temp = pd.DataFrame(data=temp, columns=columns)

In [21]:
# 예측해줍니다.

pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])['AMT'].sum().reset_index(drop=False)

In [22]:
# 디코딩 해줍니다.

temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

### 06.2. 제출용 4월 데이터 만들기

In [24]:
# 4월 데이터를 제출 데이터 형식으로 맞춰주는 작업을 합니다.

temp2 = df_2.copy()

columns = ['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']

REG_YYMMs = temp2['REG_YYMM'].unique().tolist()
CARD_SIDO_NMs = temp2['CARD_SIDO_NM'].unique().tolist()
STD_CLSS_NMs = temp2['STD_CLSS_NM'].unique().tolist()

temp_zero = []
for REG_YYMM in REG_YYMMs:
    for CARD_SIDO_NM in CARD_SIDO_NMs:
        for STD_CLSS_NM in STD_CLSS_NMs:
            temp_zero.append([REG_YYMM, CARD_SIDO_NM, STD_CLSS_NM])

temp_zero = pd.DataFrame(data=temp_zero, columns=columns)

In [25]:
# 실제 4월 데이터와 합쳐주기.

temp_group = temp2.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])['AMT'].sum().reset_index(drop=False)
temp_ = pd.merge(temp_zero, temp_group, how='left', on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])

In [26]:
# 결측치는 0원으로 채워줍니다.

temp_ = temp_.fillna(0)

### 06.3. 4월, 7월 합치고 제출용 파일 만들기

In [27]:
# 4월 데이터와 7월 예측 데이터와 합쳐줍니다.

temp_final = pd.concat([temp, temp_])

In [28]:
# 최종 제출파일 만들어 줍니다.

submission = sub.copy()
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp_final,
                              left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'],
                              right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'],
                              how='left')
submission.index.name = 'id'
submission.to_csv('submission_final.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,8.882399e+07
1,202004,강원,골프장 운영업,4.708347e+09
2,202004,강원,과실 및 채소 소매업,1.121029e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.436078e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
